In [1]:
from datetime import date
from glob import glob
import json
import math
import os
import sys
import time

import gspread
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
sys.path.append(r'C:\Users\lesliec\code')

In [3]:
from tbd_eeg.tbd_eeg.data_analysis.eegutils import EEGexp
from tbd_eeg.tbd_eeg.data_analysis.Utilities.utilities import find_nearest_ind

In [4]:
%matplotlib notebook

#### Load Zap_Zip-log_exp to get metadata for experiments

In [5]:
_gc = gspread.service_account() # need a key file to access the account
_sh = _gc.open('Templeton-log_exp') # open the spreadsheet
_df = pd.DataFrame(_sh.sheet1.get()) # load the first worksheet
metadata = _df.T.set_index(0).T # put it in a nicely formatted dataframe

### Load subjects

In [6]:
subjects = {
    '631037': {
        'awake-saline': r'F:\psi_exp\mouse631037\estim_2022-12-06_09-54-04\experiment1\recording1',
    },
    '654182': {
        'awake-saline': r'F:\psi_exp\mouse654182\estim_vis_2022-12-01_10-33-50\experiment1\recording1',
    },
    '655956': {
        'awake-saline': r'F:\psi_exp\mouse655956\estim_2022-12-15_10-07-59\experiment1\recording1',
    },
    '657903': {
        'awake-psilocybin': r'F:\psi_exp\mouse657903\pilot_aw_psi_2023-01-13_12-18-22\experiment1\recording1',
    },
    '666193': {
        'saline': r'F:\psi_exp\mouse666193\pilot_aw_2023-02-15_11-44-11\experiment1\recording1',
        'psilocybin': r'F:\psi_exp\mouse666193\pilot_aw_psi_2023-02-16_10-55-48\experiment1\recording1',
    }, 
    '666194': {
        'saline': r'F:\psi_exp\mouse666194\pilot_aw_2023-02-22_12-32-58\experiment1\recording1',
        'psilocybin': r'F:\psi_exp\mouse666194\pilot_aw_psi_2023-02-23_10-40-34\experiment1\recording1',
    },
    '666196': {
        'saline': r'F:\psi_exp\mouse666196\pilot_aw_2023-03-15_12-29-06\experiment1\recording1',
        'psilocybin': r'F:\psi_exp\mouse666196\pilot_aw_psi_2023-03-16_10-21-29\experiment1\recording1',
    },
    '669118': {
        'saline': r'F:\psi_exp\mouse669118\pilot_aw_2023-03-23_12-14-39\experiment1\recording1',
        'psilocybin': r'F:\psi_exp\mouse669118\pilot_aw_psi_2023-03-24_09-55-33\experiment1\recording1',
    },
    '669117': {
        'saline': r'F:\psi_exp\mouse669117\pilot_aw_2023-03-29_11-09-15\experiment1\recording1',
        'psilocybin': r'F:\psi_exp\mouse669117\pilot_aw_psi_2023-03-30_11-37-07\experiment1\recording1',
    },
    '673449': {
        'psilocybin1': r'F:\psi_exp\mouse673449\aw_psi_2023-04-19_11-23-26\experiment1\recording1',
        'psilocybin2': r'F:\psi_exp\mouse673449\aw_psi_d2_2023-04-20_10-05-31\experiment1\recording1',
        'saline': r'F:\psi_exp\mouse673449\aw_2023-04-21_09-28-23\experiment1\recording1',
    },
    '676726': {
        'psilocybin': r'F:\psi_exp\mouse676726\aw_psi_2023-05-03_11-08-22\experiment1\recording1',
    },
    '676727': {
        'psilocybin': r'F:\psi_exp\mouse676727\aw_psi_2023-05-10_09-49-12\experiment1\recording1',
    },
    '678912': {
        'psilocybin': r'P:\mouse678912\spont_aw_psi_2023-06-22_11-42-00\experiment1\recording1',
    },
    '678913': {
        'psilocybin': r'P:\mouse678913\spont_aw_psi_2023-06-29_12-49-40\experiment1\recording1',
    },
}

### Load EEGexp

In [7]:
mouse = '678913'
exptype = 'psilocybin' # 'saline', 'psilocybin', 'awake-saline', 'awake-psilocybin'
exp = EEGexp(subjects[mouse][exptype], preprocess=False, make_stim_csv=False)
print(exp.experiment_data)
print(exp.date)

Experiment type: sensory stimulation
['probeB_sorted', 'probeC_sorted', 'probeD_sorted', 'probeF_sorted', 'recording1']
2023-06-29 12:49:40


In [8]:
msg_file = sorted(glob(exp.data_folder + '/**/text.npy', recursive=True))[0]
msg = np.load(msg_file).astype(str)
print(msg)

msgSN_file = sorted(glob(os.path.dirname(msg_file) + '/sample_numbers.npy', recursive=True))[0]
msgSN = np.load(msgSN_file)

['saline Injection' 'saline Injection' 'psilocybin injection '
 'psilocybin injection ']


In [10]:
if (len(exp.experiment_data) == 1) and (np.any([True for x in exp.experiment_data if 'recording' in x])):
    print('Using EEG timestamps...')
    message_samples = msgSN
    masterts = np.load(exp.ephys_params['EEG']['timestamps'])
    mastersamples = np.load(os.path.join(os.path.dirname(exp.ephys_params['EEG']['timestamps']), 'sample_numbers.npy'))
elif np.any([True for x in exp.experiment_data if 'probe' in x]):
    print('Using NPX-LFP timestamps...')
    choose_probe = [x for x in exp.experiment_data if 'probe' in x][0].replace('_sorted', '')
    message_samples = (
        msgSN * (exp.ephys_params[choose_probe]['lfp_sample_rate'] / exp.ephys_params[choose_probe]['ap_sample_rate'])
    ).astype(int)
    masterts = np.load(exp.ephys_params[choose_probe]['lfp_timestamps'])
    mastersamples = np.load(
        os.path.join(os.path.dirname(exp.ephys_params[choose_probe]['lfp_timestamps']), 'sample_numbers.npy')
    )

Using NPX-LFP timestamps...


In [11]:
msg_ts = masterts[np.array([find_nearest_ind(mastersamples, x) for x in message_samples])]
print(msg_ts)

[2608.99499978 2608.99499978 3232.18182929 3232.20742921]


### Store injection times

In [ ]:
injection_times = {
    '631037': {
        'awake-saline': [2197.0, 2789.0],
    },
    '654182': {
        'awake-saline': [2747.0, 3365.0],
    },
    '655956': {
        'awake-saline': [2345.0, 2992.0],
    },
    '657903': {
        'awake-psilocybin': [1200.0, 1893.71],
    },
    '666193': {
        'saline': [145.52, 801.43],
        'psilocybin': [211.30, 815.56],
    }, 
    '666194': {
        'saline': [325.28, 963.74],
        'psilocybin': [197.21, 763.06],
    },
    '666196': {
        'saline': [899.0, 1547.0],
        'psilocybin': [779.0, 1443.0],
    },
    '669118': {
        'saline': [794.0, 1377.0],
        'psilocybin': [756.0, 1406.0],
    },
    '669117': {
        'saline': [862.20, 1469.86],
        'psilocybin': [868.89, 1555.26],
    },
    '673449': {
        'psilocybin1': [1207.78, 1838.93],
        'psilocybin2': [1149.93, 1837.79],
        'saline': [1212.32, 1815.98],
    },
    '676726': {
        'psilocybin': [1794.31, 2416.78],
    },
    '676727': {
        'psilocybin': [1825.0, 2490.73],
    },
    '678912': {
        'psilocybin': [2521.53, 3182.36],
    },
    '678913': {
        'psilocybin': [2608.99, 3232.18],
    },
}